# Test con Example 6.7 de Kassimali

Del libro "Matrix Analysis of Structures", 2nd. Ed. Aslam Kassimali

Determine the joint displacements, member local end forces, and support reactions for the two-story frame, subjected to a wind loading, shown in Fig.

<img src="kex67.png" width=500>

## 1. Modelo analítico

Para la carga de datos solo será necesaria la enumeración de nudos y barras, el resto de enumeraciones el programa lo hará automáticamente.

<img src="kex67_model.png" width=500>

## 2. Librerías

In [1]:
import math
import numpy as np # type: ignore

# Ubicación de la carpeta 'struktura'
import sys
sys.path.append(r"C:\Users\fgrv\OneDrive\Documentos\PythonProjects\struktura")

import analisa.edeb as edb

## 3. Datos de nudos y barras

### 3.1 Coordenadas

Las coordenadas de los nudos se pueden introducir en diccionarios, indicando el nudo (en cualquier orden) y sus coordenadas. También se pueden usar listas de coordendas, este caso los datos deben introducirse en el orden de enumeración de los nudos.

In [2]:
# Datos de los nudos
coords = {1: (0,0), 2: (9, 0), 3: (0, 6), 4: (9, 6), 5: (0, 12)}  # Coordenadas

### 3.2 Restricciones

Las restricciones de los apoyos se indican en diccionarios, con el número de nudo y sus restricciones. La cantidad de restricciones por nudo, depende de los grados de libertad del modelo, en el caso de pórticos tenemos 3 grados de libertad (horizontal, vertical, giro). Por lo que las restriccciones se deben indicar de la siguiente manera:

- Empotramiento: (1, 1, 1)
- Apoyo de segundo género: (1, 1, 0)
- Apoyo de primer género (libre en x, restringido en y): (0, 1, 0)

In [3]:
# Restricciones de apoyos
restricciones = {1: (1,1,1), 2: (1,1,1)}

### 3.3 Materiales
Los materiales se definen en la clase 'Material()' de 'edeb.py'

In [4]:
# Materiales
E = 30e9
m = edb.Material(E)

### 3.4 Secciones
Las secciones se establecen con la clase 'Seccion()' de 'edeb.py'

In [5]:
# Datos de las secciones
A = 75e-3  # área
I = 4.8e-4  # inercia
s = edb.Seccion(area=A, inercia_z=I)

### 3.5 Tabla de conectividad

Como todas las barras son del mismo material y sección, no hace falta indicar estos datos en la tabla de conectividad. Es suficiente indicando el nudo inicial y final de cada barra.

In [6]:
# Conectividad
datos_barras = {1: (1,3), 2: (2,4), 3:(3,5), 4: (3,4), 5: (4,5)}

## 4. Cargas

### 4.1 Cargas nodales

In [7]:
# Cargas exteriores en los nudos
cargas_nodales = {3: (80e3, 0, 0), 5: (40e3, 0, 0)}

### 4.2 Carga distribuida

Se consideran solo cargas uniformemente distribuidas en toda la longitud de la barra, ya sea en dirección $x$ o en dirección $y$ del sistema de coordenadas local.

- Si la carga distribuida no se extiende en toda la longidud del elemento, se debe agregar un nudo en el inicio y fin de la carga.
- Si la carga fuese variable, se deben agregar nudos en distancias razonables como para poder asumir carga uniformemente distribuida.

In [8]:
# Cargas externas sobre barras
ang = math.atan(6/9)
q = 12e3
qx = q*math.sin(ang)
qy = q*math.cos(ang)
cargas_barras = {5: (qx, qy)}

## 5. Procesamiento

### 5.1 Creación del modelo analítico

In [9]:
# Modelo analítico con todos los datos necesarios
p = edb.Portico(datos_nudos=coords, restricciones=restricciones,
            datos_barras=datos_barras, materiales=m, secciones=s,
            cargas_nodales=cargas_nodales, cargas_barras=cargas_barras)

### 5.2 Ejecución de los cálculos

In [10]:
# Cálculo de la estructura
p.procesamiento()

## 6. Resultados

### 6.1 Resultados de Kassimali

- Desplazamientos nodales (página 313)
$$
d = \begin{bmatrix}
0.185422 \, \text{m}\\
0.000418736 \, \text{m} \\
-0.0176197 \, \text{rad}\\
0.18552 \, \text{m}\\
-0.000130738 \, \text{m}\\
-0.0260283 \, \text{rad}\\
0.186622 \, \text{m}\\
0.000713665 \, \text{m}\\
0.0178911 \, \text{rad}
\end{bmatrix}
$$

- Fuerzas nodales en coordenadas locales (páginas 314 al 317)

$$
Q_1 = \begin{bmatrix}
-157.03 \, \text{kN}\\
106.05 \, \text{kN}\\
360.44 \, \text{kNm}\\
157.03 \, \text{kN}\\
-106.05 \, \text{kN}\\
275.86 \, \text{kNm}
\end{bmatrix}; \quad Q_2 = \begin{bmatrix}
49.027 \, \text{kN}\\
85.948 \, \text{kN}\\
320.31 \, \text{kNm}\\
-49.027 \, \text{kN}\\
-85.948 \, \text{kN}\\
195.38 \, \text{kNm}
\end{bmatrix}; \quad Q_3 = \begin{bmatrix}
-110.6 \, \text{kN}\\
1.6114 \, \text{kN}\\
-80.392 \, \text{kNm}\\
110.6 \, \text{kN}\\
-1.6114 \, \text{kN}\\
90.06 \, \text{kNm}
\end{bmatrix}; \quad
Q_4 = \begin{bmatrix}
-24.5 \, \text{kN}\\
-46.429 \, \text{kN}\\
-195.47 \, \text{kNm}\\
24.5 \, \text{kN}\\
46.429 \, \text{kN}\\
-222.38 \, \text{kNm}
\end{bmatrix}; \quad Q_5 = \begin{bmatrix}
93.398 \, \text{kN}\\
59.07 \, \text{kN}\\
27.004 \, \text{kNm}\\
-93.398 \, \text{kN}\\
70.73 \, \text{kN}\\
-90.061 \, \text{kNm}
\end{bmatrix}
$$

- Reacciones (página 311):
$$
R = \begin{bmatrix} -106.05 \, \text{kN} \\ -157.03 \, \text{kN} \\ 360.44 \, \text{kNm} \\ -85.948 \, \text{kN} \\ 49.027 \, \text{kN} \\ 320.31 \, \text{kNm} \end{bmatrix}
$$

### 6.2 Resultados nuestros

#### 6.2.1 Desplazamientos nodales

In [11]:
# Desplazamientos nodales
p.desplaz_gdl()

array([ 1.85423653e-01,  4.18738947e-04, -1.76198747e-02,  1.85521414e-01,
       -1.30738947e-04, -2.60284808e-02,  1.86623368e-01,  7.13669896e-04,
        1.78911956e-02])

#### 6.2.2 Fuerzas nodales en coordenadas locales

In [12]:
# Barra 1
b1 = p.barras()[0]
b1.fuerza_local()*1e-3

array([-157.02710519,  106.05122292,  360.44136804,  157.02710519,
       -106.05122292,  275.86596946])

In [13]:
# Barra 2
b2 = p.barras()[1]
b2.fuerza_local()*1e-3

array([ 49.02710519,  85.94877708, 320.31468527, -49.02710519,
       -85.94877708, 195.37797722])

In [14]:
# Barra 3
b3 = p.barras()[2]
b3.fuerza_local()*1e-3

array([-110.59910588,    1.61094215,  -80.39374239,  110.59910588,
         -1.61094215,   90.05939531])

In [15]:
# Barra 4
b4 = p.barras()[3]
b4.fuerza_local()*1e-3

array([ -24.44028077,  -46.42799931, -195.47222706,   24.44028077,
         46.42799931, -222.37976671])

In [16]:
# Barra 5
b5 = p.barras()[4]
b5.fuerza_local()*1e-3

array([ 93.29097262,  59.07024524,  27.00178948, -93.29097262,
        70.72960068, -90.05939531])

#### 6.2.3 Reacciones

In [17]:
# Reacciones
p.reacciones()*1e-3

array([-106.05122292, -157.02710519,  360.44136804,  -85.94877708,
         49.02710519,  320.31468527])